# Toronto, CA Neighborhood Venue Cluster Analysis

# Part I:  Getting the postal codes from Wikipedia

## Here we import all the needed libraries and install any packages we need

In [1]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

print('Libraries imported.')

Libraries imported.


In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

## Webscrape code adapated from labs

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc, 'html.parser')

tr_list = soup.find('table', class_='wikitable').find_all('tr')
column_names = ['PostalCode', 'Borough', 'Neighborhood']
wiki_df = pd.DataFrame(columns=column_names)

for tr in tr_list[1:]:
    td_list = [td for td in tr.contents if td != '\n']
    
    postal_code = td_list[0].text.replace('\n','')
    borough = td_list[1].text.replace('\n','')
    neighborhood = td_list[2].text.replace('\n','')
    
    wiki_df = wiki_df.append({'PostalCode': postal_code,
               'Borough': borough,
               'Neighborhood': neighborhood
              }
              ,ignore_index=True
             )
    
wiki_df.head(15)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


##  We may need this later to make sure the clean frame is the right shape

In [ ]:
arr=wiki_df["PostalCode"].unique()
len(arr)

 ## This cell tosses out any "Not assigned" data.  It doens't really need to check the Neighborhoods,
## the first sweep through Borough gets all the instances

In [5]:
wiki_df2=wiki_df
indexNames = wiki_df2[wiki_df2['Borough'] == 'Not assigned'].index
indexNames2= wiki_df2[wiki_df2['Neighborhood']=='Not assigned'].index

wiki_df2.drop(indexNames, inplace=True)
wiki_df2.drop(indexNames2, inplace=True)
wiki_df2=wiki_df2.reset_index(drop=True)
wiki_df2.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


##  This is the last step of the first part, We need to
##  make sure dataframe is the size we expected

In [6]:
wiki_df2.shape

(103, 3)

# Part 2:  Adding in the Geospatial Data

##      Pull the lat, long coordinates as provided 

In [7]:
geo_file = 'https://cocl.us/Geospatial_data'
geo_df = pd.read_csv(geo_file)
geo_df.rename(columns={"Postal Code":"PostalCode"},inplace=True)  #need this to match the previous dataframe

## Merge the Postal Code, Borough, Neighborhood, Lattitude, and Longitude

In [8]:
clean_df = pd.merge(wiki_df2,geo_df, on='PostalCode')
clean_df.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## One last check to be sure the size is correct  (Needs to be (103,5))

In [9]:
clean_df.shape

(103, 5)

# Part 3:  Clustering

In [10]:
CLIENT_ID= ''
CLIENT_SECRET = ''
VERSION = '20180604'
LIMIT = 50

## Pull lat, long for later map

In [11]:
address = 'Toronto,CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


## Suggested focus from assignment

In [12]:
tor_df = clean_df[clean_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

tor_df.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


## Check What's Left in New Dataframe

In [13]:
tor_df.shape

(39, 5)

In [14]:
tor_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Toronto

# add a red circle marker to represent the center.
folium.features.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='red',
    popup='Region Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(tor_map)

# add the restaurants as blue circle markers
for lat, lng in zip(tor_df.Latitude, tor_df.Longitude):
    folium.features.CircleMarker(
        [lat, lng],
        radius=15,
        color='green',
        popup='Neighborhood Center',
        fill = True
        
    ).add_to(tor_map)

# display map
tor_map

## Function from Lab

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## We loop through all the neighborhoods to look for venues

In [16]:
tor_venues = getNearbyVenues(names=tor_df['Neighborhood'],
                                   latitudes=tor_df['Latitude'],
                                   longitudes=tor_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


## Make sure they pulled into the dataframe the correct types

In [17]:
tor_venues.dtypes

Neighborhood               object
Neighborhood Latitude     float64
Neighborhood Longitude    float64
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
Venue Category             object
dtype: object

In [18]:
tor_venues.shape

(1185, 7)

In [19]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 216 uniques categories.


## Another function from the labs

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Mimic lab to pull in dataframes

In [20]:
# one hot encoding
tor_oh = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_oh['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_oh.columns[-1]] + list(tor_oh.columns[:-1])
tor_oh = tor_oh[fixed_columns]

tor_group = tor_oh.groupby('Neighborhood').mean().reset_index()
tor_group

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.020000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.020000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.040000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.020000,0.000000,...,0.020000,0.02,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,0.000000,...,0.000000,0.00,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
num_top_venues = 5

for hood in tor_group['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_group[tor_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.04
2   Cheese Shop  0.04
3        Bakery  0.04
4          Café  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3       Nightclub  0.08
4     Yoga Studio  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.14
1      Farmers Market  0.07
2       Garden Center  0.07
3                Park  0.07
4         Pizza Place  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Airport Service  0.20
1   Airport Lounge  0.13
2         Boutique  0.07
3            Plane  0.07
4    Boat or Ferry  0.07


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.16
1   Sandwich Place  0.06
2  Bubble Tea S

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_group['Neighborhood']

for ind in np.arange(tor_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Beer Bar,Café,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Clothing Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Bar,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Fast Food Restaurant,Farmers Market,Burrito Place,Comic Shop,Pizza Place,Gym / Fitness Center,Skate Park,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Airport,Airport Food Court
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio,Japanese Restaurant,Ice Cream Shop,Seafood Restaurant


## Setup k-means clustering

In [26]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Farmers Market,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Sushi Restaurant,Diner,Japanese Restaurant,Italian Restaurant,Sandwich Place,Portuguese Restaurant,Café,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Cosmetics Shop,Clothing Store,Middle Eastern Restaurant,Ramen Restaurant,Theater,Bookstore,Electronics Store,New American Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Cosmetics Shop,Coffee Shop,Seafood Restaurant,Creperie,Hotel,Restaurant,Park,Farmers Market,Gastropub
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analysis of Clustering

##  Looks like this is primarily food places in this cluster

In [32]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Farmers Market,Performing Arts Venue
1,Downtown Toronto,0,Coffee Shop,Yoga Studio,Sushi Restaurant,Diner,Japanese Restaurant,Italian Restaurant,Sandwich Place,Portuguese Restaurant,Café,Beer Bar
2,Downtown Toronto,0,Coffee Shop,Café,Cosmetics Shop,Clothing Store,Middle Eastern Restaurant,Ramen Restaurant,Theater,Bookstore,Electronics Store,New American Restaurant
3,Downtown Toronto,0,Café,Cosmetics Shop,Coffee Shop,Seafood Restaurant,Creperie,Hotel,Restaurant,Park,Farmers Market,Gastropub
4,East Toronto,0,Trail,Pub,Health Food Store,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
5,Downtown Toronto,0,Coffee Shop,Bakery,Beer Bar,Café,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Clothing Store
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio,Japanese Restaurant,Ice Cream Shop,Seafood Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports
8,Downtown Toronto,0,Café,Coffee Shop,Steakhouse,Hotel,Concert Hall,American Restaurant,Restaurant,Noodle House,Breakfast Spot,Smoke Shop
9,West Toronto,0,Bakery,Pharmacy,Music Venue,Gym / Fitness Center,Bus Stop,Café,Bar,Bank,Supermarket,Middle Eastern Restaurant


## A bit more variety, but only one Borough

In [34]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Tennis Court,Wine Shop,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


## More Central Toronto Specialty Venues

In [35]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Garden,Wine Shop,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


##  Similar to last 2 but boutique retail.

In [36]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Jewelry Store,Mexican Restaurant,Trail,Sushi Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


##  Looks like more "activities" than "restaurants"

In [37]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,4,Park,Trail,Playground,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
